In [ ]:
import numpy as np
from nbodykit.lab import transform
from utils import load_boss_data

from galactic_wavelets.cosmology import GalaxyCatalogScatteringOp

In [ ]:
import dask
import fitsio
import nbodykit
import scipy
from nbodykit.lab import FITSCatalog

print("Dask version:", dask.__version__)
print("Numpy version:", np.__version__)
print("Nbodykit version:", nbodykit.__version__)
print("Scipy version:", scipy.__version__)
print("Fitsio version:", fitsio.__version__)
data = FITSCatalog("data/galaxy_DR12v5_LOWZ_South.fits")
print(data['Z'].compute())

We first download and load the BOSS DR12 catalog (LOWZ South) following [nbodykit cookbook](https://nbodykit.readthedocs.io/en/latest/cookbook/boss-dr12-data.html).

In [ ]:
data, randoms, cosmo = load_boss_data()

In [ ]:
# from nbodykit.lab import FITSCatalog
# data = FITSCatalog("data/galaxy_DR12v5_LOWZ_South.fits")

In [ ]:
print(len(data['Z'].compute()))
print(len(randoms['Z'].compute()))

In [ ]:
# We estimate the center of the BOSS data survey by taking averages of the ra/dec/z ranges
randra_np = randoms['RA'].compute()
randdec_np = randoms['DEC'].compute()
randz_np = randoms['Z'].compute()
randra_np[randra_np > 180] -= 360 # To make RA in [-180, 180]

In [ ]:
# Determine a line of sight from the RA/DEC/z ranges
mean_ra = (randra_np.max() + randra_np.min()) / 2
mean_dec = (randdec_np.max() + randdec_np.min()) / 2
mean_z = (randz_np.max() + randz_np.min()) / 2
radecz_center = transform.SkyToCartesian(np.array([mean_ra]), np.array([mean_dec]), np.array([mean_z]), cosmo=cosmo).compute()[0]
los = radecz_center / np.linalg.norm(radecz_center) # The los goes from (0, 0, 0) to radecz_center

print("Line of sight: ", los)

In [ ]:
# Determine size and center of the box
rand_positions_np = randoms['Position'].compute()
box_size = [rand_positions_np[:, i].max() - rand_positions_np[:, i].min() for i in range(3)]
box_size = [box_size[i] * 1.05 for i in range(3)] # Add 5% to the box size to be safe for the actual data
box_center = [(rand_positions_np[:, i].max() + rand_positions_np[:, i].min()) / 2 for i in range(3)]

print("Size of the box: ", box_size)
print("Center of the box: ", box_center)

In [ ]:
kc = 4/3*np.pi
J = 6
Q = 2
angular_width = np.pi/4
scattering = True
kmax = 0.5

device = 0

In [ ]:
wst_op = GalaxyCatalogScatteringOp(J=J,
                                      Q=Q,
                                      kc=kc,
                                      angular_width=angular_width,
                                      scattering=scattering,
                                      kmax=kmax,
                                      box_size=box_size,
                                      box_center=box_center,
                                      los=los,
                                      los_auto_detection=False,
                                      device=device)

In [ ]:
s0, s1, s2 = wst_op(data, randoms=randoms)